In [9]:
import pandas as pd
import numpy as np
import warnings; 
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')
import h2o
h2o.init(max_mem_size = "5G")
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

ModuleNotFoundError: No module named 'h2o'

In [2]:
flag_early = 0
if flag_early:
    tag_pred = 'early'
else:
    tag_pred = 'late'

In [4]:
# LOAD AND MERGE DATA FILES

# load
path = '/data/WorkData/spatialtemporal/'
file = pd.read_csv(path + 'finalmerge_any_' + tag_pred + 'v2.csv')
timefile = pd.read_csv(path + 'gr_lastyear_' + tag_pred + '.csv')
test_cases = pd.read_csv(path + 'test_cases_last_hearingv2.csv', header=None)

# merge
file = pd.merge(file, timefile, on=['idnproceeding','idncase'], how='left')

In [5]:
test_cases = test_cases.rename(columns={0:'num'})
train = file[~file.idncase.isin(test_cases.num)]
test = file[file.idncase.isin(test_cases.num)]

In [7]:
train.count()

idncase                            438696
idnproceeding                      438696
dec                                438696
numAppsPerProc                     438696
nat                                438696
case_type                          438696
c_asy_type                         438696
base_city_code                     438696
hearing_loc_code                   438696
osc_date                           438696
comp_date                          438696
attorney_flag                      438696
tracid                             438696
hearing_city                       438696
base_city                          438696
numProcPerCase                     438696
adj_medium                         438696
adj_date                           438696
sched_type                         438696
lang_hearing                       438696
notice_desc                        207300
adj_time_start2                    207300
adj_time_stop2                     207300
durationHearing                   

In [8]:
# columns to drop in transformer depending on the model
if flag_early: 
    cols = ['idncase', 'idnproceeding','adj_date','comp_date','osc_date','base_city_code', 'hearing_loc_code'
            'notice_desc','adj_time_start2','adj_time_stop2', 'caseDuration',
            'numAppsPerProc','numProcPerCase','osc_date','adj_rsn_desc', 'durationFirstLastHearing', 'averageHearingDur'
            'numHearingsPerProc']
else: 
    cols = ['idncase', 'idnproceeding','adj_date','comp_date','osc_date','numAppsPerProc','numProcPerCase','base_city_code',
            'hearing_loc_code', 'notice_desc','adj_time_start2','adj_time_stop2','adj_rsn_desc']

In [ ]:
class Cleaning(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # drop unused columns        
        X = X.drop(columns=cols)
        # changes frame to H2O compatible frame
        X = h2o.H2OFrame(X)
        # encodes as categorical
        X['dec'] = X['dec'].asfactor()
        return X
    

In [ ]:
clean = Cleaning()
train = clean.transform(train)

In [ ]:
# specifying which columns to use as x and y 
training_columns = train.columns.values.tolist()[1:]
response_column = 'dec'

In [ ]:
model = H2ORandomForestEstimator(ntrees=100, max_depth=20, nfolds=5, keep_cross_validation_predictions= True, stopping_metric='auc')
model.train(x=training_columns, y=response_column, training_frame=train)`

In [ ]:
model